In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from lxml import html

from PIL import Image
import io
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains

import requests
import time
import csv
import re 
import os

from multiprocessing import Pool
import numpy as np
import pandas as pd
import random
import json

In [2]:
with open('data.json', 'r') as fp:
    data = json.load(fp)

In [3]:
data

{'Outdoors & Recreation': ['Wild Pacific Trail',
  'Giant Cedar Trail',
  'Clayoquot Plateau Provincial Park',
  'Muriel Lake',
  'Brigade Lake',
  'Wanetta Lake',
  'Angus Lake',
  'Angora Lake',
  'Rathtrevor Beach Provincial Park',
  'Stocking Creek Trail',
  'Englishman River Falls',
  'Little Qualicum Falls Provincial Park',
  'Englishman River Falls Provincial Park',
  'Stamp River Provincial Park',
  'Sproat Lake Provincial Park',
  'Mount Arrowsmith',
  'Cathedral Grove',
  'Newcastle Island Marine Provincial Park',
  'Mount Benson',
  'McCoy Lake',
  'Mount Cokely ',
  'Hole in the Wall',
  'MacMillan Provincial Park',
  'Cameron Lake',
  'Horne Lake',
  'The Bastion',
  'Shelton Lake',
  'Fourth Lake',
  'Rhododendron Lake',
  'Loudon Park and Trail',
  'Nitobe Memorial Garden',
  'Kitsilano Beach',
  'Cape Flattery',
  'French Beach Provincial Park',
  'Pacific Rim National Park Reserve',
  'Juan de Fuca Provincial Park',
  'Cape Flattery Trail',
  'Carmanah Walbran Provinci

In [4]:
data.keys()

dict_keys(['Outdoors & Recreation', 'Motoring', 'Camping', 'Sports', 'Popular'])

In [5]:
names = []

for i in data['Outdoors & Recreation']:
    if names.count(i) == 0:
        names.append(i)
    
for i in data['Camping']:
    if names.count(i) == 0:
        names.append(i)

for i in data['Sports']:
    if names.count(i) == 0:
        names.append(i)

for i in data['Motoring']:
    if names.count(i) == 0:
        names.append(i)

for i in data['Popular']:
    if names.count(i) == 0:
        names.append(i)

len(names)

2317

In [46]:
path_chrome = r"D:\chromedriver"
url  = 'https://maps.roadtrippers.com/?lng=-126.35144998641863&lat=55.342552909447306&z=4.320481614698682'

driver = webdriver.Chrome(path_chrome)
driver.get(url)
print("{} got loaded".format(url))

time.sleep(10)
for i in range(3):
    driver.find_element_by_xpath('//*[@id="trip-creation"]/section[{}]/div/span'.format(i+1)).click()

element = driver.find_element_by_xpath('//*[@id="search-text"]')
search  = driver.find_element_by_xpath('//*[@id="search"]/div[3]/div[1]')
element.clear()
element.send_keys(names[0])
search.click()

try:
    time.sleep(40)
    driver.find_element_by_xpath('//*[@id="layer-panel"]/div/div/div/ul/li[1]/div/div/a/div[1]').click()

except:
    print('not found')
    driver.close()
    
try:
    driver.find_element_by_xpath('//*[@id="place"]/div[4]/div[2]/div[1]/div[1]/div/div[2]').click()
except:
    pass

time.sleep(10)
soup = bs(driver.page_source, 'html.parser')

description = {}
try:
    desc = soup.find('div', class_='description')
    description['title'] = desc.find('h2').text

    desc = soup.find('div', class_='description-content')
    description['title'] = desc.find('p').text
except:
    pass

try:
    timings = soup.find('section', class_='hours')
    timings = timings.find('li').text
    print(timings)
except:
    timings = None
    
urls = []
try:
    driver.find_element_by_xpath('//*[@id="place"]/div[1]/div[2]/div[3]/div/div[1]/div/a/div/div[2]').click()
    time.sleep(2)

    print('#######3')
    driver.find_element_by_xpath('//*[@id="gallery"]/div/div/div[1]/div/div[4]').click()
    time.sleep(10)
    print('#######3')
    soup = bs(driver.page_source, 'html.parser')
    imgs = soup.find_all('a', class_='js-route image-hover')

    print(len(imgs))

    for i in range(min(5, len(imgs))):
        img = imgs[i].find('div', class_='image loaded')
        print(img)
        img = img['style']
        urls.append(img)

    driver.back()
    driver.back()
    time.sleep(3)

except:
    pass

try:
    driver.find_element_by_xpath('//*[@id="place"]/div[4]/div[2]/div[2]/div[4]/div/div[2]/div').click()
    time.sleep(3)
    soup = bs(driver.page_source, 'html.parser')

    general = soup.find('section', class_='general-attributes')

    try:
        general_no = general.find_all('li', class_='false')
    except:
        general_no = []

    try:
        general_unk = general.find_all('li', class_='unknown')
    except:
        general_unk = []

    try:
        general = general.find_all('li')
    except:
        general = []

    set1 = set(general_no)
    set2 = set(general_unk)
    set3 = set(general)
    res = list(set3 - set1)
    res

    Tags = []
    for i in res:
        Tags.append(i.text)

    general = soup.find('div', class_='secondary-attributes')

    try:
        general_no = general.find_all('li', class_='false')
    except:
        general_no = []

    try:
        general_unk = general.find_all('li', class_='unknown')
    except:
        general_unk = []

    try:
        general = general.find_all('li')
    except:
        general = []

    set1 = set(general_no)
    set2 = set(general_unk)
    set3 = set(general)
    res = list(set3 - set1)
    res

    for i in res:
        Tags.append(i.text)

    print(Tags)

except:
    Tags = []

driver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[2]').click()
time.sleep(10)

https://maps.roadtrippers.com/?lng=-126.35144998641863&lat=55.342552909447306&z=4.320481614698682 got loaded
Sun - Sat: 12:00 am - 11:59 pm
#######3
#######3
3
<div class="image loaded" data-rt-lazy-loading="true" style='background-image: url("https://atlas-assets.roadtrippers.com/uploads/place_image/image/1026889474/-strip_-quality_60_-interlace_Plane_-resize_376x376_U__-gravity_center_-extent_376x376/place_image-image-a143c5a8-6b9a-410e-98e4-cd05134d483a.jpg");'></div>
<div class="image loaded" data-rt-lazy-loading="true" style='background-image: url("https://atlas-assets.roadtrippers.com/uploads/place_image/image/1026889475/-strip_-quality_60_-interlace_Plane_-resize_376x376_U__-gravity_center_-extent_376x376/place_image-image-4975f600-6030-49ae-a71c-f1286f7adeba.jpg");'></div>
<div class="image loaded" data-rt-lazy-loading="true" style='background-image: url("https://atlas-assets.roadtrippers.com/uploads/place_image/image/1026889476/-strip_-quality_60_-interlace_Plane_-resize_376x3

In [47]:
soup = bs(driver.page_source, 'html.parser')
adres = soup.find('p', class_='address')
adres = adres.text.replace('\n',' ')

In [48]:
x = driver.current_url.split('?')[1]
lng = x.split('&')[0].split('lng=')[1]
lat = x.split('&')[1].split('lat=')[1]

In [49]:
lng

'-125.5622359461504'

In [50]:
lat

'48.923758623926574'

In [51]:
social = soup.find('div', class_='social')
mail = social.find('a')['href'].split('mailto:')[1]
site = social.find('a', class_='website')['href']

In [52]:
mail

'wildpacifictrail@gmail.com'

In [17]:
from util_ import scrape_site
if __name__== "__main__":
    p = Pool(2)
    p.map(scrape_site, names[:2])

# name = names[0:2]

In [18]:
df = pd.read_csv('_tag.csv')
df

,Name,Tags,Timings,Images,Description,Site,Mail,Address,Latitude,Longitude
0,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
1,Wild Pacific Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
2,Wild Pacific Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
3,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
4,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
5,Wild Pacific Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
6,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
7,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
8,Wild Pacific Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145
9,Giant Cedar Trail British Columbia,[],NaN,[],{},NaN,NaN,NaN,55.342553,-126.35145


In [6]:
from util import scrape_gmap
if __name__=='__main__':
    p = Pool(2)
    p.map(scrape_gmap, names[:2])

ModuleNotFoundError: No module named 'cssutils'

In [15]:
df = pd.read_csv('gmap.csv')
df

,Name,Rating,Services,Tags,Timings,Coordinates,Comments,Details,Address
0,Clayoquot Plateau Provincial Park British Colu...,4.8,None,[],{},"['49.2323763', '-125.4187276']","[{'title': 'TimHoppe-Carrerra', 'rate': ' 5 st...","['Ucluelet, BC V0R 3A0, Canada', 'bcparks.ca',...","Ucluelet, BC V0R 3A0, Canada"
1,Wild Pacific Trail British Columbia,4.8,None,[],{},"['48.9385887', '-125.5629913']","[{'title': 'NickWilson', 'rate': ' 5 stars ', ...",[],"Ucluelet, BC V0R 3A0, Canada"
2,Giant Cedar Trail British Columbia,4.7,None,"['beautiful', 'legs', 'stretch', 'highway', 's...",{'Tuesday': ' 8am–4pm Holiday hours Hours mig...,"['51.1070085', '-117.9087419']","[{'title': 'RussellMayne', 'rate': ' 5 stars '...","['Trans Canada Hwy, Columbia-Shuswap B, BC, Ca...","Trans Canada Hwy, Columbia-Shuswap B, BC, Canada"


In [17]:
df.iloc[0]['Comments']

"[{'title': 'TimHoppe-Carrerra', 'rate': ' 5 stars ', 'info': 'For those that know, timing is everything ,mid July brings a feast of multiple wild berries and plenty of Sun filled days to allow you to enjoy the waterfalls and small pools to bathe in. Yes the water is cold but NOT as cold as Tofino and …'}, {'title': 'LloydKenny', 'rate': ' 4 stars ', 'info': 'The water is really cold but very refreshing. A great stop on the way to Tofino.'}, {'title': 'JaimeTavares', 'rate': ' 5 stars ', 'info': 'Beautiful scenery, there are many places to go to be alone and look at the nice views. There is an abundance of places to go to just see nature and embrace this beautiful country we live in.'}, {'title': 'RowanSpitz', 'rate': ' 5 stars ', 'info': 'Beautiful views'}, {'title': 'TylerFerguson', 'rate': ' 5 stars ', 'info': 'Beautiful rock formations and emerald water, just stopped on the roadside pullout for what was supposed to be a minute and spent almost forty-five there.'}, {'title': 'JPeder